2018/05/01

打ち合わせ資料

櫻岡響

# 列番号からの文字列推定実験
## RDBの使用により以下が実現
- sentenceをid指定で呼び出し
- 言語モデルとsentenceがidで紐付いている
- 生成した言語モデルにチェックサムが付いているので、後から再現可能（現状バイナリで保存しているので16進数文字列に変更予定）

# 準備

In [26]:
from functools import reduce
from sklearn.model_selection import train_test_split

from anakin.lang_model.lang_model import LangModel

from anakin.lang_model_file_factory import LangModelFileFactory
from anakin.db.session import database, engine_and_session
from anakin.db.model import Base, Sentence, File

from anakin.util.word import Word
from anakin.util.estimate import estimate
from anakin.util.key_to_word import KeyToWord
from anakin.util.dp_matching import DPMatching

from env import LOCAL

import pandas as pd
import numpy as np
from matplotlib import pyplot
import seaborn as sns

In [2]:
# import warnings
# warnings.filterwarnings('ignore')

In [3]:
db = database(**LOCAL)
ENGINE, Session = engine_and_session(db, echo=False)
Base.prepare(ENGINE, reflect=True)

lmff = LangModelFileFactory(db, echo=False)

/Users/hbk/.anyenv/envs/pyenv/versions/miniconda3-latest/lib/python3.6/site-packages/sqlalchemy/ext/declarative/clsregistry.py:120: SAWarning: This declarative base already contains a class with the same class name and module name as sqlalchemy.ext.automap.datasets, and will be replaced in the string-lookup table.
  item.__name__
/Users/hbk/.anyenv/envs/pyenv/versions/miniconda3-latest/lib/python3.6/site-packages/sqlalchemy/ext/declarative/clsregistry.py:120: SAWarning: This declarative base already contains a class with the same class name and module name as sqlalchemy.ext.automap.migrate_version, and will be replaced in the string-lookup table.
  item.__name__


In [16]:
def id_to_vocab(sentences):   
    result = set()
    for sentence in sentences:
        con = sentence.contents
        words = Word.from_sentence(con)
        result = result | set(words)
    return KeyToWord(result)

In [17]:
def id_to_lm(lang_model_file_factory, sentences, order):
    id_list = [s.id for s in sentences]
    lmff = lang_model_file_factory
    lmf = lmff.get_lang_model_file(id_list, order)
    arpa = lmf.contents.decode()
    return lmf.checksum, LangModel(arpa)

# 対象ファイル名検索

In [6]:
session = Session()
q = session.query(File.file_name).all()
target_file_name = q[0][0]
session.close()
target_file_name

'travel02_userReview00_20160304.txt'

In [10]:
session = Session()
file = session.query(File).filter(File.file_name == target_file_name).one()
datum = [data for data in file.datum]
sentences = [sentence for data in datum for sentence in data.sentences]
sentences

[<Sentence(id=1, contents='この/コノ ホテル/ホテル で/デ この/コノ 値段/ネダン は/ハ 安い/ヤスイ'),
 <Sentence(id=2, contents='駅/エキ から/カラ 出/デ て/テ すぐ/スグ の/ノ 所/トコロ に/ニ あり/アリ まし/マシ た/タ'),
 <Sentence(id=3, contents='雨/アメ でも/デモ 安心/アンシン です/デス ね/ネ'),
 <Sentence(id=4, contents='乗車/ジョウシャ 券/ケン が/ガ 有効/ユウコウ 利用/リヨウ でき/デキ まし/マシ た/タ'),
 <Sentence(id=5, contents='バス/バス ルーム/ルーム が/ガ 広い/ヒロイ の/ノ が/ガ よかっ/ヨカッ た/タ です/デス'),
 <Sentence(id=6, contents='ラウンジ/ラウンジ で/デ コーヒー/コーヒー を/ヲ 頼ん/タノン だら/ダラ プチケーキ/プチケーキ が/ガ 付い/ツイ て/テ き/キ た/タ'),
 <Sentence(id=7, contents='お/オ 土産/ミヤゲ を/ヲ 買う/カウ とき/トキ に/ニ 便利/ベンリ です/デス ね/ネ'),
 <Sentence(id=8, contents='ウォシュレット/ウォシュレット 付/ヅケ'),
 <Sentence(id=9, contents='食事/ショクジ が/ガ おいしかっ/オイシカッ た/タ です/デス'),
 <Sentence(id=10, contents='<eng> 地下鉄/チカテツ とも/トモ 駅/エキ に/ニ 近く/チカク 時計/トケイ 台/ダイ も/モ そば/ソバ'),
 <Sentence(id=11, contents='ただし/タダシ すすきの/ススキノ に/ニ は/ハ 遠い/トオイ'),
 <Sentence(id=12, contents='歩い/アルイ て/テ <num> 分/フン ほど/ホド で/デ コンビニ/コンビニ 酒/シュ も/モ あり/アリ が/ガ ある/アル の/ノ は/ハ 嬉しい/ウレシイ'),
 <Sentence(id=13, contents='シャンプー/シャンプー ひ/ヒ げ/ゲ そり/ソリ 歯ブラ

# 実験条件

In [18]:
order = 3
test_num = 100
seed = 12345678

# 実験条件をもとにトレーニングデータとテストデータに分ける

In [52]:
size = len(sentences)
size

2582976

In [53]:
train, test = train_test_split(sentences, test_size=test_num/size, random_state=seed)

close_train = sentences
open_train = train

In [83]:
def exec_test(test_setences, vocab, lm, order, max_=None):
    res = []
    for idx, sentence in enumerate(test):
        if not max_ is None and idx == max_ + 1:
            break
        ref = Word.from_sentence(sentence.contents)
        est = estimate(ref, vocab, lm, order)
        dpm = DPMatching(ref, est)

        print(idx, dpm.score)
        
        series = pd.Series({
            'ref':' / '.join([w.surface for w in ref]),
            'ref_size':len(ref),
            'est':' / '.join([w.surface for w in est]),
            'perfect':dpm.score.perfect,
            'yomi':dpm.score.yomi,
            'miss':dpm.score.miss
        })
        res.append(series)
    return pd.DataFrame(
        res,
        columns=['ref','est', 'ref_size', 'perfect', 'yomi', 'miss']
    )

# クローズドテスト

In [21]:
close_train_vocab = id_to_vocab(close_train)
checksum, close_train_lm = id_to_lm(lmff, close_train, order)

/Users/hbk/.anyenv/envs/pyenv/versions/miniconda3-latest/lib/python3.6/site-packages/pymysql/cursors.py:165: Warning: (3170, "Memory capacity of 8388608 bytes for 'range_optimizer_max_mem_size' exceeded. Range optimization was not done for this query.")
  result = self._query(query)
/Users/hbk/.anyenv/envs/pyenv/versions/miniconda3-latest/lib/python3.6/site-packages/pymysql/cursors.py:165: Warning: (1300, "Invalid utf8 character string: '87534E'")
  result = self._query(query)


In [84]:
result_close = exec_test(test, close_train_vocab, close_train_lm, order, max_=None)
result_close

0 Score(perfect=16, yomi=1, miss=0, ignore=False)
1 Score(perfect=16, yomi=0, miss=0, ignore=False)
2 Score(perfect=15, yomi=0, miss=0, ignore=False)
3 Score(perfect=21, yomi=0, miss=0, ignore=False)
4 Score(perfect=11, yomi=0, miss=0, ignore=False)
5 Score(perfect=4, yomi=0, miss=0, ignore=False)
6 Score(perfect=8, yomi=0, miss=0, ignore=False)
7 Score(perfect=13, yomi=0, miss=0, ignore=False)
8 Score(perfect=11, yomi=1, miss=0, ignore=False)
9 Score(perfect=7, yomi=0, miss=-1, ignore=False)
10 Score(perfect=43, yomi=1, miss=0, ignore=False)
11 Score(perfect=29, yomi=0, miss=0, ignore=False)
12 Score(perfect=13, yomi=0, miss=0, ignore=False)
13 Score(perfect=15, yomi=0, miss=0, ignore=False)
14 Score(perfect=6, yomi=0, miss=-3, ignore=False)
15 Score(perfect=10, yomi=0, miss=0, ignore=False)
16 Score(perfect=14, yomi=1, miss=-1, ignore=False)
17 Score(perfect=16, yomi=0, miss=0, ignore=False)
18 Score(perfect=23, yomi=0, miss=0, ignore=False)
19 Score(perfect=44, yomi=1, miss=-1, igno

,ref,est,ref_size,perfect,yomi,miss
0,多少 / たばこ / の / 臭い / が / 気 / に / なっ / た / くらい /...,多少 / タバコ / の / 臭い / が / 気 / に / なっ / た / くらい /...,17,16,1,0
1,禁煙 / ルーム / の / 設定 / を / し / て / も / 良い / の / で...,禁煙 / ルーム / の / 設定 / を / し / て / も / 良い / の / で...,16,16,0,0
2,車 / で / の / 宿泊 / に / なる / の / です / が / 駐車 / 場 ...,車 / で / の / 宿泊 / に / なる / の / です / が / 駐車 / 場 ...,16,15,0,0
3,ホテル / は / 昔 / の / 旅館 / という / 感じ / で / ノスタルジック ...,ホテル / は / 昔 / の / 旅館 / という / 感じ / で / ノスタルジック ...,21,21,0,0
4,設備 / は / 少し / 古 / め / です / が / 手入れ / が / 行き届い ...,設備 / は / 少し / 古 / め / です / が / トイレ / が / 行き届い ...,12,11,0,0
5,の / 中 / から / お / 好み / の / 枕 / を / 選べる,何 / か / カラオケ / の / 目 / の / 枕 / を / 選べる,9,4,0,0
6,あまりの / も / 簡単 / に / 予約 / でき / まし / た / ので,あまりに / も / 簡単 / に / 予約 / でき / まし / た / ので,9,8,0,0
7,浅草 / セントラル / ホテル / 様 / の / おかげ / で / いい / 旅 / ...,浅草 / セントラル / ホテル / 様 / の / おかげ / で / いい / 旅 / ...,13,13,0,0
8,厚木 / の / 宿泊 / で / は / 必ず / 利用 / さ / せ / て / 頂い...,お / 得 / な / 宿泊 / で / は / 必ず / 利用 / さ / せ / て /...,14,11,1,0
9,ありがとう / ござい / ます / とても / 快適 / な / 滞在 / でし / た,ありがとう / ござい / まし / た / でも / 快適 / な / 滞在 / でし / た,9,7,0,-1


In [85]:
with open('result_close.csv', 'w') as f:
    f.write(result_close.to_csv())

# オープンテスト

In [ ]:
%%time
open_train_vocab = id_to_vocab(open_train)
checksum, open_train_lm = id_to_lm(lmff, open_train, order)